# CSV Cleanup and JSON Conversion

This notebook processes the app store digital health app reviews CSV file to:
1. Keep only app_name, title, score (rating), and review columns
2. Filter out non-English content
3. Convert the data to a specific JSON format

In [8]:
# Import necessary libraries
import pandas as pd
import json
import re
import langdetect
from langdetect import detect
from tqdm.notebook import tqdm

## Step 1: Load and Examine the CSV file

In [9]:
# Load the CSV file
df = pd.read_csv('../data/interim/app_store_digital_health_app_reviews.csv')

# Display the first few rows
print(f"Total number of reviews: {len(df)}")
df.head()

Total number of reviews: 3600


,date,review,rating,isEdited,title,userName,developerResponse,app_name,app_id
0,2019-08-04 20:08:12,A great service providing a platform for succe...,5,False,App Review,SSJ Klahr,NaN,MyFitnessPal,341232718
1,2022-04-24 17:36:05,I really like this app. I use it for calorie c...,4,False,Great app but could use more user control of s...,georg-o,"{'id': 29434530, 'body': 'Thank you for provid...",MyFitnessPal,341232718
2,2020-06-29 17:51:29,I’ve been using this app for years. There are ...,4,False,Gross Pictures with Ads but extensive database,AliHart1983,"{'id': 28070018, 'body': 'Hi there! Our apolog...",MyFitnessPal,341232718
3,2024-06-21 09:54:51,I really love this app! It has helped me in so...,5,False,Love it! Have some suggestions:,Bmhz92,"{'id': 44810091, 'body': 'Thank you for taking...",MyFitnessPal,341232718
4,2022-10-03 20:41:42,"Several years ago, this app was the best out t...",2,False,No longer a good app,Artigius,"{'id': 32395408, 'body': 'We are truly sorry f...",MyFitnessPal,341232718


## Step 2: Clean and Filter the Data

In [10]:
# Select only the columns we need
df_clean = df[['app_name', 'title', 'rating', 'review']].copy()

# Rename 'rating' to 'score' for clarity
df_clean = df_clean.rename(columns={'rating': 'score'})

# Drop rows with missing values
df_clean = df_clean.dropna()

# Reset index
df_clean = df_clean.reset_index(drop=True)

# Display the first few rows of cleaned data
print(f"Remaining reviews after cleaning: {len(df_clean)}")
df_clean.head()

Remaining reviews after cleaning: 3600


,app_name,title,score,review
0,MyFitnessPal,App Review,5,A great service providing a platform for succe...
1,MyFitnessPal,Great app but could use more user control of s...,4,I really like this app. I use it for calorie c...
2,MyFitnessPal,Gross Pictures with Ads but extensive database,4,I’ve been using this app for years. There are ...
3,MyFitnessPal,Love it! Have some suggestions:,5,I really love this app! It has helped me in so...
4,MyFitnessPal,No longer a good app,2,"Several years ago, this app was the best out t..."


## Step 3: Filter for English Content

We'll use the langdetect library to identify and keep only English content.

In [11]:
# Helper function to detect language with error handling
def is_english(text):
    if not isinstance(text, str) or text.strip() == '':
        return False
    try:
        return detect(text) == 'en'
    except:
        return False

In [12]:
# Filter for English content only
print("Filtering for English content...")

# To make processing faster, first check titles (which are shorter)
english_title_mask = df_clean['title'].apply(is_english)
df_english = df_clean[english_title_mask].copy()

# Then check reviews (this might take a bit longer)
# Using tqdm for progress tracking
tqdm.pandas(desc="Checking reviews")
english_review_mask = df_english['review'].progress_apply(is_english)
df_english = df_english[english_review_mask].copy()

# Reset index
df_english = df_english.reset_index(drop=True)

print(f"Remaining reviews after language filtering: {len(df_english)}")
df_english.head()

Filtering for English content...


Checking reviews:   0%|          | 0/2857 [00:00<?, ?it/s]

Remaining reviews after language filtering: 2856


,app_name,title,score,review
0,MyFitnessPal,Great app but could use more user control of s...,4,I really like this app. I use it for calorie c...
1,MyFitnessPal,Gross Pictures with Ads but extensive database,4,I’ve been using this app for years. There are ...
2,MyFitnessPal,No longer a good app,2,"Several years ago, this app was the best out t..."
3,MyFitnessPal,Difficult to navigate,2,I consider myself very tech savvy and the user...
4,MyFitnessPal,Great App for someone with a disability,5,"I have Limb Girdle Muscular Dystrophy, and I a..."


## Step 4: Create the Required JSON Format

The format should be:
```
{ 
app_name1: {[title1, score1, review1], [title2, score2, review2], [title3, score3, review3]},
app_name2: {[title1, score1, review1], [title2, score2, review2], [title3, score3, review3]},
app_name3: {[title1, score1, review1], [title2, score2, review2], [title3, score3, review3]},
... 
}
```

In [17]:
# Create the JSON structure
result_dict = {}

# Group by app_name
for app_name, app_group in df_english.groupby('app_name'):
    app_reviews = []
    for _, row in app_group.iterrows():
        app_reviews.append([row['title'], row['score'], row['review']])
    
    # Add to the result dictionary
    result_dict[app_name] = app_reviews


# Display sample of the result
print(f"Total number of apps: {len(result_dict)}")
sample_app = next(iter(result_dict))
print(f"Sample app: {sample_app}")
print(f"Number of reviews for sample app: {len(result_dict[sample_app])}")
# Show the first review entry
sample_title = result_dict[sample_app][0]  # Get the first review entry
print(f"Sample title: {sample_title[0]}")  # Access the title from the list
print(f"Sample review data: {sample_title}")  # Print the entire review data

Total number of apps: 18
Sample app: 7 Minute Workout
Number of reviews for sample app: 155
Sample title: This app is amazing!
Sample review data: ['This app is amazing!', 5, 'This is one of the best apps I’ve ever gotten! And I’m not just saying that! I use it all the time and I used to do it with my parents all the time, I’m never one to get inn-app-purchases and even with out them this app is a great workout, I do a few of these a day, and they really work up a sweat and have me gasping for air at the end. It’s not on-going nor too long breaks, you can pause it if you need a longer break or need to take a quick one in-between the exercise, but I thing 10 morning minutes is the perfect amount of time to get a sip of water and prepare for the next exercise. I think this workout is effective, and it’s super quick and fun with quite a bit of variety and quick breaks in between therefore, even without the subscriptions this is probably the best workout app to use (keep in mind, this is j

## Step 5: Save to JSON File

In [ ]:
# Save the result to a JSON file
output_filename = '../data/processed/app_store_reviews.json'

with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(result_dict, f, ensure_ascii=False, indent=4)

print(f"Processed data saved to {output_filename}")

Processed data saved to ../data/processed/app_store_reviews_processed.json


## Step 6: Verify the JSON File (Optional)

In [ ]:
# Load the JSON file to verify it
with open(output_filename, 'r', encoding='utf-8') as f:
    loaded_data = json.load(f)

# Check if the loaded data matches our original result
assert len(loaded_data) == len(result_dict), "JSON file content doesn't match original data"
print(f"Verification successful. JSON file contains data for {len(loaded_data)} apps.")

# Count total number of reviews
total_reviews = sum(len(app_data) for app_data in loaded_data.values())
print(f"Total number of reviews in the final JSON: {total_reviews}")